In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/movie_data/movie_data"))

# Any results you write to the current directory are saved as output.

['full_test.txt', 'full_train.txt']


In [83]:
import numpy as np
import pandas as pd
import os
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [84]:
reviews_train = []
for line in open('../input/movie_data/movie_data/full_train.txt', 'r'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('../input/movie_data/movie_data/full_test.txt', 'r'):
    
    reviews_test.append(line.strip())
    
target = [1 if i < 12500 else 0 for i in range(25000)]

In [85]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [ ]:
baseline_vectorizer = CountVectorizer(binary=True)
baseline_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(baseline_vectorizer.get_feature_names()))
#print("Words in dictionary: ", baseline_vectorizer.get_feature_names())

In [ ]:
X_baseline = baseline_vectorizer.transform(reviews_train_clean)
X_test_baseline = baseline_vectorizer.transform(reviews_test_clean)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_baseline, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test_baseline)))

**Remove stop words**

In [ ]:
from nltk.corpus import stopwords

english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

In [ ]:
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [ ]:
stopword_vectorizer = CountVectorizer(binary=True)
stopword_vectorizer.fit(no_stop_words_train)
print("Size of dictionary: ", len(stopword_vectorizer.get_feature_names()))
#print("Words in dictionary: ", stopword_vectorizer.get_feature_names())

In [ ]:
X = stopword_vectorizer.transform(no_stop_words_train)
X_test = stopword_vectorizer.transform(no_stop_words_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        stopword_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

**Lemmatization**
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [ ]:
from nltk.stem import WordNetLemmatizer

def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

In [ ]:
lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)

In [ ]:
lemmatize_vectorizer = CountVectorizer(binary=True)
lemmatize_vectorizer.fit(lemmatized_reviews_train)
print("Size of dictionary: ", len(lemmatize_vectorizer.get_feature_names()))
#print("Words in dictionary: ", lemmatize_vectorizer.get_feature_names())

In [ ]:
X = lemmatize_vectorizer.transform(lemmatized_reviews_train)
X_test = lemmatize_vectorizer.transform(lemmatized_reviews_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        lemmatize_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

**Stemming**
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [ ]:
from nltk.stem.porter import PorterStemmer

def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

In [ ]:
stemmed_reviews_train = get_stemmed_text(reviews_train_clean)
stemmed_reviews_test = get_stemmed_text(reviews_test_clean)

In [ ]:
stemm_vectorizer = CountVectorizer(binary=True)
stemm_vectorizer.fit(stemmed_reviews_train)
print("Size of dictionary: ", len(stemm_vectorizer.get_feature_names()))
#print("Words in dictionary: ", stemm_vectorizer.get_feature_names())

In [ ]:
X = stemm_vectorizer.transform(stemmed_reviews_train)
X_test = stemm_vectorizer.transform(stemmed_reviews_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        stemm_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

**Using n-grams**

In [ ]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(ngram_vectorizer.get_feature_names()))
#print("Words in dictionary: ", ngram_vectorizer.get_feature_names())

In [ ]:
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        ngram_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

**TF-IDF (term frequency-inverse term frequency)**
https://buhrmann.github.io/tfidf-analysis.html

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', analyzer='word', ngram_range=(1, 2))
tfidf_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(tfidf_vectorizer.get_feature_names()))
#print("Words in dictionary: ", tfidf_vectorizer.get_feature_names())

In [ ]:
X = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lr = LogisticRegression(C=i)
    lr.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lr.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LogisticRegression(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        tfidf_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

**Using linear Support Vector Machine**

In [ ]:
from sklearn.svm import LinearSVC

lsvc_ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
lsvc_ngram_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(lsvc_ngram_vectorizer.get_feature_names()))
#print("Words in dictionary: ", lsvc_ngram_vectorizer.get_feature_names())

In [ ]:
X = lsvc_ngram_vectorizer.transform(reviews_train_clean)
X_test = lsvc_ngram_vectorizer.transform(reviews_test_clean)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.85)
score = []
c = [0.01, 0.05, 0.25, 0.5, 1]
for i in c:
    lsvc = LinearSVC(C=i)
    lsvc.fit(X_train, y_train)
    score.append(accuracy_score(y_val, lsvc.predict(X_val)))
    print ("Accuracy for C=%s: %s" % (i, score[-1]))

In [ ]:
print("Value using for C: ", c[np.where(max(score) == score)[0][0]])
final_model = LinearSVC(C=c[np.where(max(score) == score)[0][0]])
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        lsvc_ngram_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

In [ ]:
pos = ["I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. "
       "The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary "
       "conviction, to save lives. After losing his team he takes a teaching position training the next generation of heroes. The young troubled "
       "recruit is played by Kutcher. While his scenes with the local love interest are a tad stiff and don't generate enough heat to melt butter, "
       "he compliments Costner well. I never really understood Sela Ward as the neglected wife and felt she should of wanted Costner to quit out of "
       "concern for his safety as opposed to her selfish needs. But her presence on screen is a pleasure. The two unaccredited stars of this movie "
       "are the Coast Guard and the Sea. Both powerful forces which should not be taken for granted in real life or this movie. The movie has some "
       "slow spots and could have used the wasted 15 minutes to strengthen the character relationships. But it still works. The rescue scenes are "
       "intense and well filmed and edited to provide maximum impact. This movie earns the audience applause. And the applause of my two sons."]
print("Pos prediction: {}". format(lsvc.predict(lsvc_ngram_vectorizer.transform(pos))))

In [ ]:
neg = ["We do not want to send out an e-mail with a subject line that recipient(s) feel like avoiding.""
       "Think about your own reaction on e-mails with negative subject lines;"
       "do you even feel like opening them? Example, instead of a subject line that says ‘Delay in" 
       "ABC project schedule,’ the subject line can be ‘Changes in ABC project schedule.'"]
print("Neg prediction: {}". format(lsvc.predict(lsvc_ngram_vectorizer.transform(neg))))

**Using Naive Bayes**

In [73]:
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

english_stop_words = stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

In [74]:
no_stop_words_train = remove_stop_words(reviews_train_clean)
no_stop_words_test = remove_stop_words(reviews_test_clean)

In [75]:
stopword_vectorizer = CountVectorizer(binary=True)
stopword_vectorizer.fit(no_stop_words_train)
print("Size of dictionary: ", len(stopword_vectorizer.get_feature_names()))
#print("Words in dictionary: ", stopword_vectorizer.get_feature_names())

Size of dictionary:  86357


In [76]:
X_train = stopword_vectorizer.transform(no_stop_words_train)
X_test = stopword_vectorizer.transform(no_stop_words_test)

In [77]:
from sklearn.model_selection import GridSearchCV

X_train, X_val, y_train, y_val = train_test_split(X_train, target, train_size = 0.85)

mnb = MultinomialNB()
parameters = {
    'alpha':list(range(1, 10))
}
clf = GridSearchCV(mnb, parameters, cv=5)
clf.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [79]:
print("Best parameters: ", clf.best_estimator_)
final_model = clf.best_estimator_

Best parameters:  MultinomialNB(alpha=5, class_prior=None, fit_prior=True)


In [80]:
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Final Accuracy: 0.84252


In [81]:
feature_to_coef = {
    word: coef for word, coef in zip(
        stopword_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

('one', -5.511019995866638)
('film', -5.523028145495035)
('movie', -5.5252117358478365)
('like', -5.789023154218585)
('good', -5.915076944702976)
('time', -6.006001424253158)
('well', -6.022730184042363)
('see', -6.037215116963731)
('story', -6.045102481796896)
('great', -6.049353586399379)



('____________________________________', -12.606415993120331)
('____insert', -12.606415993120331)
('_a', -12.606415993120331)
('_absolute', -12.606415993120331)
('_am_', -12.606415993120331)
('_and_', -12.606415993120331)
('_angel_', -12.606415993120331)
('_annie_', -12.606415993120331)
('_any_', -12.606415993120331)
('_anything_', -12.606415993120331)


**Using Naive Bayes with TFIDF**

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', analyzer='word', ngram_range=(1, 2))
tfidf_vectorizer.fit(reviews_train_clean)
print("Size of dictionary: ", len(tfidf_vectorizer.get_feature_names()))
#print("Words in dictionary: ", tfidf_vectorizer.get_feature_names())

Size of dictionary:  1777206


In [87]:
X_train = tfidf_vectorizer.transform(reviews_train_clean)
X_test = tfidf_vectorizer.transform(reviews_test_clean)

In [88]:
from sklearn.model_selection import GridSearchCV

X_train, X_val, y_train, y_val = train_test_split(X_train, target, train_size = 0.85)

mnb = MultinomialNB()
parameters = {
    'alpha':list(range(1, 10))
}
clf = GridSearchCV(mnb, parameters, cv=5)
clf.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [89]:
print("Best parameters: ", clf.best_estimator_)
final_model = clf.best_estimator_

Best parameters:  MultinomialNB(alpha=1, class_prior=None, fit_prior=True)


In [90]:
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" % accuracy_score(target, final_model.predict(X_test)))

Final Accuracy: 0.85452


In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        tfidf_vectorizer.get_feature_names(), final_model.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
print("\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

In [ ]:
Naive Bayes is a very good algorithm for text classification and considered as baseline. Basically for text classification NB is benchmark where accuracy of other algorithms are compared with NB.